In [1]:
import json

with open("../data/problem_v1.1.0_20231207.json", "r", encoding="utf-8") as f:
    raw_data = json.load(f)

In [ ]:
#define the customed result files and parameters here...

details_filelist = [""]
details_weights = [1]
assert len(details_filelist) == len(details_weights)

In [ ]:
detailed_score_ensemble_list = []
for file in details_filelist:
    with open(file,"r",encoding='utf-8') as f:
        data = json.load(f)
        detailed_score_ensemble_list.append(data)
    
def judge_score_for_a_problem(problem_key):
    sumed = 0
    for det, weight in zip(detailed_score_ensemble_list, details_weights):
        prob_det = det[problem_key]
        sumed += 1-prob_det["score"]/prob_det["total_score"]*weight
    return sumed

In [8]:
select_type_categories = "yw sx yy zz ls dl wl hx sw dx jk xc".split()

classified_list = [
    [[] for _ in range(len(select_type_categories))] for _ in range(3)
]  # (0,1,m)

# First Iteration: Generate metadata of all the problems
# metadata of each question: (transferred_id, type, model_score_criteria, comp result of diff & quality)
for key, problem in raw_data.items():
    difficulty, quality = problem["difficulty"], problem["quality"]
    for i, typ in enumerate(problem["problem_type_list"]):
        meta_triplet = (
            f"{key}_{i}",
            typ,
            judge_score_for_a_problem(f"{key}_{i}"),
            difficulty * 10 + quality,
        )
        question_images = len(problem.get("img_paths", []))
        if question_images > 2:
            question_images = 2

        sub = key.split("_")[0]
        subs = "dx" if sub[:2] == "dx" else sub[-2:]

        classified_list[question_images ^ 1][select_type_categories.index(subs)].append(
            meta_triplet
        )  # [SinglePics, NonPics, MultiPics]

print(len(raw_data))

17234


In [5]:
# Multiple images list

multiple_image_list = classified_list[-1]
len(multiple_image_list)

622

In [32]:
from typing import List, Tuple, Union
import random

type_list = ["单选", "多选", "填空"]


def sorting_criteria(x: Tuple[Union[int, str]]) -> int:
    try:
        return (
            (4 - type_list.index(x[-3])) * 10000 + x[-2] * 100 + x[-1]
        )  # (Single > Multi > FillB >> others)
    except:
        return -1


import math

# selection_matrix = [[20, 22, 24], [10, 10, 12], [10, 10, 14]]
selection_matrix = [[20, 22, 24], [20, 20, 24], [20, 20, 24]] # number matrix. A precise selection will make the selection more accurate.


result_list = []

for row_index, (selection_row, metadatas_list) in enumerate(zip(selection_matrix, classified_list)):
    for i, metadata_list in enumerate(metadatas_list):
        random.shuffle(metadata_list)
        metadata_list.sort(key=sorting_criteria, reverse=True)
        sc_total, mc_total, bf_total = [
            len(list(filter(lambda x: x[-3] == type_list[k], metadata_list)))
            for k in range(3)
        ]

        # [:single_total]
        neg_index = i - len(select_type_categories)
        if neg_index < -3:  # middleschool
            j = 0
        elif neg_index == -3:  # university
            j = 1
        else:
            j = 2
        tot_needed_in_this_section = selection_matrix[row_index][j]
        mc, bf = (
            math.ceil(tot_needed_in_this_section / 4),
            math.floor(tot_needed_in_this_section / 4),
        )

        # A better solution is splitting the list into 3 and perform sampling respectively.

        result_list.extend(
            metadata_list[sc_total : sc_total + min(mc, mc_total)]
        )  # MuC
        result_list.extend(
            metadata_list[sc_total + mc_total : sc_total + mc_total + min(bf, bf_total)]
        )  # BF
        result_list.extend(
            metadata_list[
                : min(
                    sc_total,
                    tot_needed_in_this_section - min(mc, mc_total) - min(bf, bf_total),
                )
            ]
        )  #SiC

len(result_list)

604

In [35]:
random.shuffle(result_list)
result_list=result_list[:500]

with open("selected_hard_20231212.json","w",encoding='utf-8') as fw:
    json.dump(result_list,fw,ensure_ascii=False,indent=4)

In [3]:
import json

with open("../data/selected_hard_20231212.json","r",encoding='utf-8') as f:
    result_list = json.load(f)

In [7]:
with open("../data/selected_hard_list_20231212.json", "w", encoding="utf-8") as fw:
    json.dump(list(map(lambda x: x[0], result_list)), fw)

In [4]:
# Analysis on the correctness
import os

keys = map(lambda x: x[0], result_list)
# keys = result_list

input_file = (
    "../results/moss_input_0_shot_0_20231208_080359/prediction.json"
)
with open(input_file, "r", encoding="utf-8") as f:
    preds = json.load(f)

hard_performance = {}
cnt = 0
for key in keys:
    if key in preds:
        hard_performance.update({key: preds[key]})
    else:
        # print(f"{key} missig in the preds")
        cnt += 1
        
print(cnt)

output_dir = f"../results/selected_analysis/{input_file.split('/')[-2]}"
os.makedirs(output_dir, exist_ok=True)
output_file = os.path.join(output_dir, input_file.split("/")[-1])
with open(output_file, "w", encoding="utf-8") as fw:
    json.dump(hard_performance, fw, ensure_ascii=False, indent=4)

0
